In [1]:
import gc
import pandas as pd
import numpy as np
import os
import xgboost as xgb
import lightgbm as lgb
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.externals import joblib
from imblearn.over_sampling import SMOTE 
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve 

In [2]:
def score(y,pred): 
    fpr, tpr, thresholds = roc_curve(y, pred, pos_label=1) 
    score=0.4*tpr[np.where(fpr>=0.001)[0][0]]+0.3*tpr[np.where(fpr>=0.005)[0][0]]+0.3*tpr[np.where(fpr>=0.01)[0][0]] 
    return score 

In [2]:
####################训练集#######################
DATA_PATH = "datasets"
csv_path = os.path.join(DATA_PATH, "atec_anti_fraud_train.csv")
data = pd.read_csv(csv_path)

In [3]:
####################训练集#######################
pd.set_option('display.max_columns',2000)
data

,id,label,date,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,f119,f120,f121,f122,f123,f124,f125,f126,f127,f128,f129,f130,f131,f132,f133,f134,f135,f136,f137,f138,f139,f140,f141,f142,f143,f144,f145,f146,f147,f148,f149,f150,f151,f152,f153,f154,f155,f156,f157,f158,f159,f160,f161,f162,f163,f164,f165,f166,f167,f168,f169,f170,f171,f172,f173,f174,f175,f176,f177,f178,f179,f180,f181,f182,f183,f184,f185,f186,f187,f188,f189,f190,f191,f192,f193,f194,f195,f196,f197,f198,f199,f200,f201,f202,f203,f204,f205,f206,f207,f208,f209,f210,f211,f212,f213,f214,f215,f216,f217,f218,f219,f220,f221,f222,f223,f224,f225,f226,f227,f228,f229,f230,f231,f232,f233,f234,f235,f236,f237,f238,f239,f240,f241,f242,f243,f244,f245,f246,f247,f248,f249,f250,f251,f252,f253,f254,f255,f256,f257,f258,f259,f260,f261,f262,f263,f264,f265,f266,f267,f268,f269,f270,f271,f272,f273,f274,f275,f276,f277,f278,f279,f280,f281,f282,f283,f284,f285,f286,f287,f288,f289,f290,f291,f292,f293,f294,f295,f296,f297
0,f10eb20f31cf7063ee8bdbd1272214e4d7e0193c8dbce4...,0,20171103,0,0,0,0,100807.0,0,5,1,1,1,1,2,0,2,0,2,2,2,1,31.0,61.0,142.0,245.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.0,1.0,0.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,3.0,96.0,38.0,301.0,302.0,302.0,302.0,301.0,302.0,40.0,79.0,40.0,79.0,2.0,3.0,4.0,3.0,41.0,80.0,96.5,721.1,3252.0,154.5,394.5,2.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,4.0,8.0,15.0,30.0,4.0,14.0,34.0,70.0,142.0,3.0,15.0,41.0,79.0,191.0,192.0,190.0,190.0,190.0,190.0,191.0,192.0,191.0,190.0,190.0,190.0,32.0,33.0,25.0,25.0,24.0,25.0,31.0,79.0,42.0,42.0,42.0,68.0,32.0,32.0,28.0,27.0,28.0,27.0,32.0,235.0,235.0,126.0,126.0,127.0,126.0,235.0,192.0,191.0,190.0,189.0,189.0,301.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,3.0,3.0,4.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,4.0,3.0,4.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,3.0,3.0,3.0,14.0,41.0,80.0,167.0,2.0,2.0,4.0,3.0,3.0,4.0,6.0,6.0,2.0,2.0,4.0,3.0,2.0,2.0,3.0,4.0,3.0,3.0,4.0,1.0,3.0,3.0,4.0,2.0,2.0,2.0,2.0,8.0,2.0,2.0,3.0,3.0,3.0,2.0,5.0,5.0,10.0,16.0,2.0,2.0,4.0,5.0,7.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,27.0,297.0,302.0,384.0,770.0,20.0,99.0,102.0,124.0,140.0,301.0,312.0,328.0,85.0,302.0,201.0,203.0,203.0,61.0,201.0
1,d861929b67938d06538b910b9f6b85f5eb62b6ad7361ba...,0,20170917,0,1,1,1,100805.0,1,5,2,1,1,0,2,1,2,2,1,1,2,0,14.0,13.0,48.0,48.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,1.0,1.0,2.0,4.0,4.0,5.0,1.0,1.0,4.0,3.0,3.0,79.0,12.0,302.0,32.0,301.0,302.0,302.0,302.0,23.0,26.0,23.0,26.0,2.0,2.0,3.0,4.0,23.0,26.0,5.0,717.4,733.3,231.8,472.8,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,4.0,8.0,11.0,12.0,4.0,12.0,23.0,26.0,27.0,5.0,12.0,24.0,27.0,122.0,121.0,16.0,121.0,121.0,122.0,121.0,122.0,121.0,16.0,121.0,122.0,32.0,31.0,13.0,27.0,26.0,27.0,32.0,79.0,40.0,39.0,39.0,63.0,31.0,31.0,15.0,28.0,29.0,29.0,31.0,231.0,232.0,17.0,99.0,99.0,98.0,232.0,121.0,122.0,17.0,121.0,121.0,302.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,4.0,5.0,12.0,24.0,26.0,27.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,3.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1

In [4]:
####################测试集#######################
csv_path = os.path.join(DATA_PATH, "atec_anti_fraud_test_b.csv")
data_test = pd.read_csv(csv_path)

In [5]:
data_test

,id,date,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,f119,f120,f121,f122,f123,f124,f125,f126,f127,f128,f129,f130,f131,f132,f133,f134,f135,f136,f137,f138,f139,f140,f141,f142,f143,f144,f145,f146,f147,f148,f149,f150,f151,f152,f153,f154,f155,f156,f157,f158,f159,f160,f161,f162,f163,f164,f165,f166,f167,f168,f169,f170,f171,f172,f173,f174,f175,f176,f177,f178,f179,f180,f181,f182,f183,f184,f185,f186,f187,f188,f189,f190,f191,f192,f193,f194,f195,f196,f197,f198,f199,f200,f201,f202,f203,f204,f205,f206,f207,f208,f209,f210,f211,f212,f213,f214,f215,f216,f217,f218,f219,f220,f221,f222,f223,f224,f225,f226,f227,f228,f229,f230,f231,f232,f233,f234,f235,f236,f237,f238,f239,f240,f241,f242,f243,f244,f245,f246,f247,f248,f249,f250,f251,f252,f253,f254,f255,f256,f257,f258,f259,f260,f261,f262,f263,f264,f265,f266,f267,f268,f269,f270,f271,f272,f273,f274,f275,f276,f277,f278,f279,f280,f281,f282,f283,f284,f285,f286,f287,f288,f289,f290,f291,f292,f293,f294,f295,f296,f297
0,661ae9a03c6c74feab2555c9987e140ae3f5421bd8e7a0...,20180305,0,0,1,1,100810.0,0,4,2,0,1,0,2,1,1,1,0,2,2,0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,1.0,1.0,1.0,2.0,1.0,2.0,3.0,2.0,2.0,2.0,1.0,3.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,1.0,3.0,3.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,3.0,2.0,2.0,3.0,3.0,2.0,3.0,2.0,3.0,3.0,4.0,3.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,3.0,5.0,5.0,10.0,24.0,35.0,46.0,1.0,2.0,2.0,4.0,3.0,4.0,11.0,11.0,2.0,1.0,2.0,2.0,2.0,1.0,3.0,4.0,2.0,2.0,3.0,2.0,1.0,3.0,4.0,2.0,2.0,8.0,7.0,7.0,2.0,1.0,2.0,2.0,3.0,2.0,3.0,7.0,10.0,15.0,2.0,1.0,3.0,4.0,9.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,78.0,302.0,302.0,324.0,353.0,44.0,150.0,149.0,156.0,155.0,301.0,301.0,302.0,192.0,301.0,218.0,219.0,219.0,153.0,218.0
1,ada98d4358e72a27cc5e92f691a87a4fd62a7cda0387e2...,20180303,0,0,1,1,100810.0,1,6,1,0,2,1,1,0,2,2,0,2,2,0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.0,27.0,4.0,6.0,6.0,9.0,24.0,51.0,78.0,51.0,78.0,4.0,6.0,7.0,9.0,55.0,82.0,686.5,14090.8,20128.8,1392.5,4811.3,2.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,4.0,8.0,15.0,22.0,8.0,23.0,56.0,83.0,109.0,9.0,23.0,52.0,80.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,25.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,4.0,6.0,9.0,23.0,53.0,80.0,107.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,0.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,7.0,23.0,25.0,5.0,6.0,2.0,1.0,1.0,2.0,2.0
2,e33675a962e5bf44d05a2b01903a4beb88a0c6385c05c6...,20180226,0,0,1,1,100811.0,1,2,1,0,1,0,

In [6]:
cols = list(data.columns)[3:]
print(cols)

['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100', 'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108', 'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118', 'f119', 'f120', 'f121', 'f122', 'f123', 'f124', 'f125', 'f126', 'f127', 'f128', 'f129', 'f130', 'f131', 'f132', 'f133', 'f134', 'f135', 'f136', 'f137', 'f138', 'f1

In [7]:
col_int = []
col_float = []
for col in cols:
    #print(data[col].dtype)
    if data[col].dtype == "int64":
        col_int.append(col)
    if data[col].dtype == "float64":
        col_float.append(col)
print(col_int)
print(col_float)

['f1', 'f2', 'f3', 'f4', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19']
['f5', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100', 'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108', 'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118', 'f119', 'f120', 'f121', 'f122', 'f123', 'f124', 'f125', 'f126', 'f127', 'f128', 'f129', 'f130', 'f131', 'f132', 'f133', 'f134', 'f135', 'f136', 'f137', 'f138', 'f

In [31]:
################## 缺失90%的特征 #############################
# missing_90 = ['f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47']
# data.drop(missing_90, axis=1, inplace=True)
# data_test.drop(missing_90, axis=1, inplace=True)
# for i in missing_90:
#     cols.remove(i)
# print(len(cols))

In [8]:
##############训练集onehot编码#####################
data_onehot = []
for col in col_int:
    dummies = pd.get_dummies(data[col], prefix=col)
    data_onehot.append(dummies)
    
for i in data_onehot:
    data = pd.concat([data, i], axis=1)
    
data.drop(col_int, axis=1, inplace=True)

In [9]:
##############测试集onehot编码#####################
data_test_onehot = []
for col in col_int:
    dummies = pd.get_dummies(data_test[col], prefix=col)
    data_test_onehot.append(dummies)

for i in data_test_onehot:
    data_test = pd.concat([data_test, i], axis=1)
    
data_test.drop(col_int, axis=1, inplace=True)

In [10]:
##################缺失值填充########################
data.fillna(0, inplace=True)
data_test.fillna(0, inplace=True)

In [32]:
#################对数据进行标准化####################
sc = StandardScaler()
data[col_float] = sc.fit_transform(data[col_float])
data_test[col_float] = sc.transform(data_test[col_float])

NameError: name 'data_test' is not defined

In [11]:
data.loc[data['label'] == -1, 'label'] = 1
data.loc[data['label'] == 1, 'label'] = 2
data.loc[data['label'] == 0, 'label'] = 1
data.loc[data['label'] == 2, 'label'] = 0
data['label'].value_counts()

1    977884
0     16847
Name: label, dtype: int64

In [17]:
#data_sorted = data.sort_values("date")

In [18]:
#data_sorted

,id,label,date,f5,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,f32,f33,f34,f35,f36,f37,f38,f39,f40,f41,f42,f43,f44,f45,f46,f47,f48,f49,f50,f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f62,f63,f64,f65,f66,f67,f68,f69,f70,f71,f72,f73,f74,f75,f76,f77,f78,f79,f80,f81,f82,f83,f84,f85,f86,f87,f88,f89,f90,f91,f92,f93,f94,f95,f96,f97,f98,f99,f100,f101,f102,f103,f104,f105,f106,f107,f108,f109,f110,f111,f112,f113,f114,f115,f116,f117,f118,f119,f120,f121,f122,f123,f124,f125,f126,f127,f128,f129,f130,f131,f132,f133,f134,f135,f136,f137,f138,f139,f140,f141,f142,f143,f144,f145,f146,f147,f148,f149,f150,f151,f152,f153,f154,f155,f156,f157,f158,f159,f160,f161,f162,f163,f164,f165,f166,f167,f168,f169,f170,f171,f172,f173,f174,f175,f176,f177,f178,f179,f180,f181,f182,f183,f184,f185,f186,f187,f188,f189,f190,f191,f192,f193,f194,f195,f196,f197,f198,f199,f200,f201,f202,f203,f204,f205,f206,f207,f208,f209,f210,f211,f212,f213,f214,f215,f216,f217,f218,f219,f220,f221,f222,f223,f224,f225,f226,f227,f228,f229,f230,f231,f232,f233,f234,f235,f236,f237,f238,f239,f240,f241,f242,f243,f244,f245,f246,f247,f248,f249,f250,f251,f252,f253,f254,f255,f256,f257,f258,f259,f260,f261,f262,f263,f264,f265,f266,f267,f268,f269,f270,f271,f272,f273,f274,f275,f276,f277,f278,f279,f280,f281,f282,f283,f284,f285,f286,f287,f288,f289,f290,f291,f292,f293,f294,f295,f296,f297,f1_0,f1_1,f1_2,f2_0,f2_1,f2_2,f3_0,f3_1,f3_2,f4_0,f4_1,f4_2,f6_0,f6_1,f6_2,f6_3,f6_4,f7_0,f7_1,f7_2,f7_3,f7_4,f7_5,f7_6,f7_7,f8_0,f8_1,f8_2,f9_0,f9_1,f9_2,f10_0,f10_1,f10_2,f11_0,f11_1,f11_2,f12_0,f12_1,f12_2,f13_0,f13_1,f13_2,f14_0,f14_1,f14_2,f15_0,f15_1,f15_2,f16_0,f16_1,f16_2,f17_0,f17_1,f17_2,f18_0,f18_1,f18_2,f19_0,f19_1,f19_2
937719,ef5ad2df31fc279cf5d32f6f0d9b045232a1ae0ebaa84e...,1,20170905,100803.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,0.0,1.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,21.0,20.0,21.0,21.0,20.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,1,0,0,0,1,0,1,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0
936071,a439318d736ef43ce553b230e4524d8261dfe6877f7498...,1,20170905,100804.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,4.0,3.0,3.0,4.0,2.0,3.0,4.0,4.0,5.0,5.0,3000.0,5298.7,5299.7,5298.7,5298.7,2.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,2.0,5.0,4.0,4.0,5.0,5.0,4.0,5.0,4.0,4.0,1.0,2.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,2.0,1.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,2.0,1.0,4.0,4.0,4.0,5.0,5.0,4.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,2

In [34]:
###################数据集划分（线下）########################
off_train = data[data['date'] <= 20171017]
off_val = data[(data['date'] <= 20171027) & (data['date'] > 20171017)]
off_test = data[data['date'] > 20171027]
print(off_train.shape)
print(off_val.shape)
print(off_test.shape)

(684011, 343)
(162587, 343)
(148133, 343)


In [35]:
###################数据集划分（线下）########################
X_train = off_train.loc[:,'f5':].as_matrix()
y_train = off_train.loc[:,'label'].as_matrix()
print(X_train.shape, y_train.shape)

X_val = off_val.loc[:,'f5':].as_matrix()
y_val = off_val.loc[:,'label'].as_matrix()
print(X_val.shape, y_val.shape)

X_test = off_test.loc[:,'f5':].as_matrix()
y_test = off_test.loc[:,'label'].as_matrix()
print(X_test.shape, y_test.shape)

(684011, 340) (684011,)
(162587, 340) (162587,)
(148133, 340) (148133,)


In [36]:
###################数据集划分（线下）########################
# for i in range(len(y_test)):
#     if y_test[i] == 1:
#         y_test[i] = 0
#     else:
#         y_test[i] = 1

In [12]:
###################数据集划分（线上）########################
on_train = data[data['date'] <= 20171031]
on_val = data[data['date'] > 20171031]
print(on_train.shape, on_val.shape)

# data_train1 = data[data['date'] <= 20171005]
# data_val1 = data[(data['date'] > 20171005) & (data['date'] <= 20171010)]
# print(data_train1.shape, data_val1.shape)

# data_train2 = data[(data['date'] > 20170920) & (data['date'] <= 20171020)]
# data_val2 = data[(data['date'] > 20171020) & (data['date'] <= 20171025)]
# print(data_train2.shape, data_val2.shape)

# data_train3 = data[(data['date'] > 20171005) & (data['date'] <= 20171105)]
# data_val3 = data[(data['date'] > 20171031) & (data['date'] <= 20171105)]
# print(data_train3.shape, data_val3.shape)

(911606, 343) (83125, 343)


In [15]:
###################数据集划分（线上）########################
X = data.loc[:,'f5':].as_matrix()
y = data.loc[:,'label'].as_matrix()
print(X.shape, y.shape)

# X_train = on_train.loc[:,'f5':].as_matrix()
# y_train = on_train.loc[:,'label'].as_matrix()
# print(X_train.shape, y_train.shape)
# X_val = on_val.loc[:,'f5':].as_matrix()
# y_val = on_val.loc[:,'label'].as_matrix()
# print(X_val.shape, y_val.shape)

# # X_train1 = data_train1.loc[:,'f5':].as_matrix()
# # y_train1 = data_train1.loc[:,'label'].as_matrix()
# # print(X_train1.shape, y_train1.shape)
# # X_val1 = data_val1.loc[:,'f5':].as_matrix()
# # y_val1 = data_val1.loc[:,'label'].as_matrix()
# # print(X_val1.shape, y_val1.shape)

# # X_train2 = data_train2.loc[:,'f5':].as_matrix()
# # y_train2 = data_train2.loc[:,'label'].as_matrix()
# # print(X_train2.shape, y_train2.shape)
# # X_val2 = data_val2.loc[:,'f5':].as_matrix()
# # y_val2 = data_val2.loc[:,'label'].as_matrix()
# # print(X_val2.shape, y_val2.shape)

# # X_train3 = data_train3.loc[:,'f5':].as_matrix()
# # y_train3 = data_train3.loc[:,'label'].as_matrix()
# # print(X_train3.shape, y_train3.shape)
# # X_val3 = data_val3.loc[:,'f5':].as_matrix()
# # y_val3 = data_val3.loc[:,'label'].as_matrix()
# # print(X_val3.shape, y_val3.shape)

X_test = data_test.loc[:,'f5':].as_matrix()
print(X_test.shape)

(994731, 340) (994731,)
(500538, 340)


In [19]:
###################SMOTE过采样#############################
# n_sample = y_train.shape[0]
# n_pos_sample = y_train[y_train == 0].shape[0]
# n_neg_sample = y_train[y_train == 1].shape[0]
# print('样本个数：{}; 正样本占{:.2%}; 负样本占{:.2%}'.format(n_sample, n_pos_sample / n_sample, n_neg_sample / n_sample))

# sm = SMOTE(random_state=42)    # 处理过采样的方法
# X_train, y_train = sm.fit_sample(X_train, y_train)
# print('通过SMOTE方法平衡正负样本后')
# n_sample = y_train.shape[0]
# n_pos_sample = y_train[y_train == 0].shape[0]
# n_neg_sample = y_train[y_train == 1].shape[0]
# print('样本个数：{}; 正样本占{:.2%}; 负样本占{:.2%}'.format(n_sample, n_pos_sample / n_sample, n_neg_sample / n_sample))

样本个数：911606; 正样本占1.70%; 负样本占98.30%
通过SMOTE方法平衡正负样本后
样本个数：1792160; 正样本占50.00%; 负样本占50.00%


In [14]:
del on_train, on_val
gc.collect()

55

In [21]:
clf = RandomForestClassifier(
                                n_estimators=300,
                                criterion="gini",
                                max_features="auto",
                                max_depth=None,
                                min_samples_split=2,
                                min_samples_leaf=1,
                                random_state=2018,
                                verbose=2,
)
clf.fit(X, y)

KeyboardInterrupt: 

In [19]:
y_pred = clf.predict_proba(X_test)[:,0]

[Parallel(n_jobs=1)]: Done 300 out of 300 | elapsed:   14.7s finished


In [19]:
#################### LogisticRegression单模型 ###########################
clf = LogisticRegression(
                            penalty='l2',
                            C=1.0,
                            random_state=2018,
                            solver='sag',
                            max_iter=100,
                            verbose=2
                       )
clf.fit(X, y)

max_iter reached after 332 seconds


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\sag.py:326: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  5.5min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  5.5min finished


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=2018, solver='sag', tol=0.0001,
          verbose=2, warm_start=False)

In [ ]:
y_pred = clf.predict_proba(X_test)[:,0]

In [68]:
#################### lightGBM单模型 ###########################
clf = lgb.LGBMClassifier(  
                            boosting_type='gbdt',
                            num_leaves=55,
                            #max_depth=-1,
                            n_estimators=300,
                            #max_bin=50,
                            objective='binary',
                            #min_split_gain=0.3,
                            learning_rate=0.05,
                            feature_fraction=0.9,
                            bagging_fraction=0.8,
                            random_state=2018,
                            #reg_alpha=0.7,
                            #reg_lambda=0.4,
                            bagging_freq=5,
                            verbose=0,
                            n_jobs=-1     
                        )

clf.fit(
            X_train, y_train, 
            eval_set=[(X_val, y_val), (X_train, y_train)], 
            eval_metric='auc', 
            early_stopping_rounds=10,
            #feature_name=cols,
            #categorical_feature=col_int
       )

[1]	valid_0's auc: 0.958604	valid_1's auc: 0.947232
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's auc: 0.961452	valid_1's auc: 0.950045
[3]	valid_0's auc: 0.962433	valid_1's auc: 0.953206
[4]	valid_0's auc: 0.96247	valid_1's auc: 0.953642
[5]	valid_0's auc: 0.962519	valid_1's auc: 0.953263
[6]	valid_0's auc: 0.964611	valid_1's auc: 0.954949
[7]	valid_0's auc: 0.964833	valid_1's auc: 0.955084
[8]	valid_0's auc: 0.964941	valid_1's auc: 0.955051
[9]	valid_0's auc: 0.966013	valid_1's auc: 0.958209
[10]	valid_0's auc: 0.96612	valid_1's auc: 0.958308
[11]	valid_0's auc: 0.966734	valid_1's auc: 0.959328
[12]	valid_0's auc: 0.966692	valid_1's auc: 0.959711
[13]	valid_0's auc: 0.96679	valid_1's auc: 0.96023
[14]	valid_0's auc: 0.966985	valid_1's auc: 0.960628
[15]	valid_0's auc: 0.967003	valid_1's auc: 0.960791
[16]	valid_0's auc: 0.967395	valid_1's auc: 0.961042
[17]	valid_0's auc: 0.967413	valid_1's auc: 0.961251
[18]	valid_0's auc: 0.967581	valid_1's auc: 0.9615

[155]	valid_0's auc: 0.980026	valid_1's auc: 0.987695
[156]	valid_0's auc: 0.980055	valid_1's auc: 0.987765
[157]	valid_0's auc: 0.980105	valid_1's auc: 0.987804
[158]	valid_0's auc: 0.980139	valid_1's auc: 0.987893
[159]	valid_0's auc: 0.980161	valid_1's auc: 0.988013
[160]	valid_0's auc: 0.980198	valid_1's auc: 0.988062
[161]	valid_0's auc: 0.980224	valid_1's auc: 0.988104
[162]	valid_0's auc: 0.980246	valid_1's auc: 0.988152
[163]	valid_0's auc: 0.980289	valid_1's auc: 0.98822
[164]	valid_0's auc: 0.980309	valid_1's auc: 0.988303
[165]	valid_0's auc: 0.98032	valid_1's auc: 0.988441
[166]	valid_0's auc: 0.980371	valid_1's auc: 0.988494
[167]	valid_0's auc: 0.980403	valid_1's auc: 0.988594
[168]	valid_0's auc: 0.980417	valid_1's auc: 0.988629
[169]	valid_0's auc: 0.980431	valid_1's auc: 0.988694
[170]	valid_0's auc: 0.98045	valid_1's auc: 0.98877
[171]	valid_0's auc: 0.98048	valid_1's auc: 0.98885
[172]	valid_0's auc: 0.980486	valid_1's auc: 0.988902
[173]	valid_0's auc: 0.980538	vali

LGBMClassifier(bagging_fraction=0.8, bagging_freq=5, boosting_type='gbdt',
        class_weight=None, colsample_bytree=1.0, feature_fraction=0.9,
        learning_rate=0.05, max_bin=50, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=300,
        n_jobs=-1, num_leaves=55, objective='binary', random_state=2018,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=1, verbose=0)

In [69]:
y_pred = clf.predict_proba(X_test)[:,0]

In [47]:
print(score(y_test, y_pred))

0.40751445086705207


In [45]:
clf = xgb.XGBClassifier(
                            silent=0,#设置成1则没有运行信息输出，最好是设置为0.是否在运行升级时打印消息。
                            nthread=-1,# cpu 线程数 默认最大
                            learning_rate= 0.1, # 如同学习率
                            min_child_weight=1, 
                            # 这个参数默认是 1，是每个叶子里面 h 的和至少是多少，对正负样本不均衡时的 0-1 分类而言
                            #，假设 h 在 0.01 附近，min_child_weight 为 1 意味着叶子节点中最少需要包含 100 个样本。
                            #这个参数非常影响结果，控制叶子节点中二阶导的和的最小值，该参数值越小，越容易 overfitting。
                            max_depth=10, # 构建树的深度，越大越容易过拟合
                            gamma=0,  # 树的叶子节点上作进一步分区所需的最小损失减少,越大越保守，一般0.1、0.2这样子。
                            subsample=1, # 随机采样训练样本 训练实例的子采样比
                            max_delta_step=0,#最大增量步长，我们允许每个树的权重估计。
                            colsample_bytree=1, # 生成树时进行的列采样 
                            reg_lambda=1,  # 控制模型复杂度的权重值的L2正则化项参数，参数越大，模型越不容易过拟合。
                            #reg_alpha=0, # L1 正则项参数
                            #scale_pos_weight=1, #如果取值大于0的话，在类别样本不平衡的情况下有助于快速收敛。平衡正负权重
                            objective='binary:logistic', #多分类的问题 指定学习任务和相应的学习目标
                            n_estimators=100, #树的个数
                            seed=2018, #随机种子
                            eval_metric= 'auc'
)

clf.fit(
            X_train, y_train, 
            eval_set=[(X_train, y_train), (X_val, y_val)], 
            eval_metric='auc',
            early_stopping_rounds=10
    )

[0]	validation_0-auc:0.957114	validation_1-auc:0.942462
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 10 rounds.
[1]	validation_0-auc:0.961565	validation_1-auc:0.948772
[2]	validation_0-auc:0.965096	validation_1-auc:0.953245
[3]	validation_0-auc:0.965703	validation_1-auc:0.95501
[4]	validation_0-auc:0.966131	validation_1-auc:0.959902
[5]	validation_0-auc:0.968229	validation_1-auc:0.963941
[6]	validation_0-auc:0.970423	validation_1-auc:0.972796
[7]	validation_0-auc:0.970493	validation_1-auc:0.973638
[8]	validation_0-auc:0.971158	validation_1-auc:0.974583
[9]	validation_0-auc:0.971468	validation_1-auc:0.975292
[10]	validation_0-auc:0.971534	validation_1-auc:0.975757
[11]	validation_0-auc:0.971561	validation_1-auc:0.976046
[12]	validation_0-auc:0.971652	validation_1-auc:0.976319
[13]	validation_0-auc:0.971699	validation_1-auc:0.976745
[14]	validation_0-auc:0.972297	validation_1-auc:0.978047

[142]	validation_0-auc:0.979693	validation_1-auc:0.997722
[143]	validation_0-auc:0.97969	validation_1-auc:0.997797
[144]	validation_0-auc:0.979689	validation_1-auc:0.99781
[145]	validation_0-auc:0.979691	validation_1-auc:0.997814
[146]	validation_0-auc:0.979681	validation_1-auc:0.997818
[147]	validation_0-auc:0.979639	validation_1-auc:0.997828
[148]	validation_0-auc:0.979659	validation_1-auc:0.997846
[149]	validation_0-auc:0.979657	validation_1-auc:0.997953
[150]	validation_0-auc:0.979635	validation_1-auc:0.997972
[151]	validation_0-auc:0.979623	validation_1-auc:0.997977
[152]	validation_0-auc:0.979561	validation_1-auc:0.997982
[153]	validation_0-auc:0.979587	validation_1-auc:0.99799
[154]	validation_0-auc:0.979612	validation_1-auc:0.998026
[155]	validation_0-auc:0.97961	validation_1-auc:0.998029
[156]	validation_0-auc:0.979611	validation_1-auc:0.998044
[157]	validation_0-auc:0.979701	validation_1-auc:0.998073
[158]	validation_0-auc:0.979634	validation_1-auc:0.998139
[159]	validation_0

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, eval_metric='auc', gamma=0, learning_rate=0.1,
       max_delta_step=0, max_depth=10, min_child_weight=1, missing=None,
       n_estimators=200, n_jobs=1, nthread=-1, objective='binary:logistic',
       random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=1,
       seed=2018, silent=0, subsample=1)

In [203]:
#################### lightGBM单模型 ###########################
clf2 = lgb.LGBMClassifier(  
                            boosting_type='gbdt',
                            num_leaves=50,
                            n_estimators=300,
                            objective='binary',
                            learning_rate=0.05,
                            feature_fraction=0.9,
                            bagging_fraction=0.8,
                            random_state=2018,
                            bagging_freq=5,
                            verbose=0,
                            n_jobs=-1     
                        )

clf2.fit(
            X_train2, y_train2, 
            eval_set=[(X_val2, y_val2), (X_train2, y_train2)], 
            eval_metric='auc', 
            early_stopping_rounds=50,
            #feature_name=cols,
            #categorical_feature=col_int
       )

NameError: name 'X_train2' is not defined

In [28]:
#################### lightGBM单模型 ###########################
clf3 = lgb.LGBMClassifier(  
                            boosting_type='gbdt',
                            num_leaves=50,
                            n_estimators=300,
                            objective='binary',
                            learning_rate=0.05,
                            feature_fraction=0.9,
                            bagging_fraction=0.8,
                            random_state=2018,
                            bagging_freq=5,
                            verbose=0,
                            n_jobs=-1     
                        )

clf3.fit(
            X_train3, y_train3, 
            eval_set=[(X_val3, y_val3), (X_train3, y_train3)], 
            eval_metric='auc', 
            early_stopping_rounds=50,
            #feature_name=cols,
            #categorical_feature=col_int
       )

[1]	valid_0's auc: 0.947367	valid_1's auc: 0.94246
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's auc: 0.953982	valid_1's auc: 0.949861
[3]	valid_0's auc: 0.959144	valid_1's auc: 0.958208
[4]	valid_0's auc: 0.96028	valid_1's auc: 0.958672
[5]	valid_0's auc: 0.960257	valid_1's auc: 0.958933
[6]	valid_0's auc: 0.965557	valid_1's auc: 0.963382
[7]	valid_0's auc: 0.96563	valid_1's auc: 0.963668
[8]	valid_0's auc: 0.967497	valid_1's auc: 0.965236
[9]	valid_0's auc: 0.967594	valid_1's auc: 0.965423
[10]	valid_0's auc: 0.967731	valid_1's auc: 0.965462
[11]	valid_0's auc: 0.967678	valid_1's auc: 0.965624
[12]	valid_0's auc: 0.967597	valid_1's auc: 0.965554
[13]	valid_0's auc: 0.967921	valid_1's auc: 0.965843
[14]	valid_0's auc: 0.967936	valid_1's auc: 0.965896
[15]	valid_0's auc: 0.968902	valid_1's auc: 0.967565
[16]	valid_0's auc: 0.971801	valid_1's auc: 0.970028
[17]	valid_0's auc: 0.971647	valid_1's auc: 0.970032
[18]	valid_0's auc: 0.97165	valid_1's auc: 0.9700

[155]	valid_0's auc: 0.990182	valid_1's auc: 0.991421
[156]	valid_0's auc: 0.990277	valid_1's auc: 0.991521
[157]	valid_0's auc: 0.990359	valid_1's auc: 0.991589
[158]	valid_0's auc: 0.990399	valid_1's auc: 0.991642
[159]	valid_0's auc: 0.990462	valid_1's auc: 0.991686
[160]	valid_0's auc: 0.990513	valid_1's auc: 0.991747
[161]	valid_0's auc: 0.990575	valid_1's auc: 0.991777
[162]	valid_0's auc: 0.990605	valid_1's auc: 0.991823
[163]	valid_0's auc: 0.990706	valid_1's auc: 0.991872
[164]	valid_0's auc: 0.990771	valid_1's auc: 0.991925
[165]	valid_0's auc: 0.990796	valid_1's auc: 0.991988
[166]	valid_0's auc: 0.990849	valid_1's auc: 0.992028
[167]	valid_0's auc: 0.990918	valid_1's auc: 0.99207
[168]	valid_0's auc: 0.990958	valid_1's auc: 0.992123
[169]	valid_0's auc: 0.990989	valid_1's auc: 0.992163
[170]	valid_0's auc: 0.991011	valid_1's auc: 0.992189
[171]	valid_0's auc: 0.99106	valid_1's auc: 0.992273
[172]	valid_0's auc: 0.991111	valid_1's auc: 0.992311
[173]	valid_0's auc: 0.991223	

LGBMClassifier(bagging_fraction=0.8, bagging_freq=5, boosting_type='gbdt',
        class_weight=None, colsample_bytree=1.0, feature_fraction=0.9,
        learning_rate=0.05, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=300,
        n_jobs=-1, num_leaves=50, objective='binary', random_state=2018,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=1, verbose=0)

In [29]:
y_pred1 = clf1.predict_proba(X_test)[:,0]
y_pred2 = clf2.predict_proba(X_test)[:,0]
y_pred3 = clf3.predict_proba(X_test)[:,0]
y_pred = 0.2 * y_pred1 + 0.3 * y_pred2 + 0.5 * y_pred3

In [20]:
result = pd.DataFrame({'id':data_test['id'].as_matrix(), 'score':y_pred})
result.to_csv("result19.csv", index=False)

In [17]:
######################## Bagging #########################
clf = lgb.LGBMClassifier(  
                            boosting_type='gbdt',
                            num_leaves=50,
                            max_depth=-1, 
                            n_estimators=300,
                            objective='binary',
                            learning_rate=0.05,
                            feature_fraction=0.9,
                            bagging_fraction=0.8,
                            random_state=2018,
                            bagging_freq=5,
                            verbose=0,
                            n_jobs=1     
                        )

bagging = BaggingClassifier(
                                clf, 
                                n_estimators=20, 
                                max_samples=0.8, 
                                max_features=1.0,
                                bootstrap=True,
                                bootstrap_features=False,
                                n_jobs=1,
                                random_state=2018,
                                verbose=2
                           )

bagging.fit(X, y)

Building estimator 1 of 20 for this parallel run (total 20)...
Building estimator 2 of 20 for this parallel run (total 20)...
Building estimator 3 of 20 for this parallel run (total 20)...
Building estimator 4 of 20 for this parallel run (total 20)...
Building estimator 5 of 20 for this parallel run (total 20)...
Building estimator 6 of 20 for this parallel run (total 20)...
Building estimator 7 of 20 for this parallel run (total 20)...
Building estimator 8 of 20 for this parallel run (total 20)...
Building estimator 9 of 20 for this parallel run (total 20)...
Building estimator 10 of 20 for this parallel run (total 20)...
Building estimator 11 of 20 for this parallel run (total 20)...
Building estimator 12 of 20 for this parallel run (total 20)...
Building estimator 13 of 20 for this parallel run (total 20)...
Building estimator 14 of 20 for this parallel run (total 20)...
Building estimator 15 of 20 for this parallel run (total 20)...
Building estimator 16 of 20 for this parallel run

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 60.4min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 60.4min finished


BaggingClassifier(base_estimator=LGBMClassifier(bagging_fraction=0.8, bagging_freq=5, boosting_type='gbdt',
        class_weight=None, colsample_bytree=1.0, feature_fraction=0.9,
        learning_rate=0.05, max_depth=-1, min_child_samples=20,
        min_child_weight=0.001, min_split_gain=0.0, n_estimators=300,
        n_jobs=-1, num_leaves=50, objective='binary', random_state=2018,
        reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
        subsample_for_bin=200000, subsample_freq=1, verbose=0),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=0.8, n_estimators=20, n_jobs=1, oob_score=False,
         random_state=2018, verbose=2, warm_start=False)

In [25]:
joblib.dump(clf, "bagging.m")

['bagging.m']

In [24]:
y_pred = bagging.predict_proba(X_test)[:,0]
result = pd.DataFrame({'id':data_test['id'].as_matrix(), 'score':y_pred})
result.to_csv("result13.csv", index=False)

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.2min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  3.2min finished


In [26]:
clf.feature_importances_

array([268,  47,  79,  59,  87,  73,  81, 103,  76, 101,  93, 160, 145,
        80,  59,  72,  53,  16,   0,   3,   5,  21,   2,   4,   2,   5,
         3,  21,  25,  72,  62,  69,  62, 116,  65, 192,  41,  31,  38,
        66,   7,   8,  21,  38,  58,  30,  39,  66,  17,  12,  11,  16,
        14,  19,  37,  20,  33,  43,  46,  64,  67,  75, 142, 329, 137,
       128, 225, 120,  23,  26,  20,  36,  35,  20,  23,  24,  43,  34,
        69,  38,  44,  42, 110,  56,  50,  55, 101, 238,  25,  23,  19,
        74,  16,   9,  15,   8,   9,  10,   4,  12,  17,  16,  17,   5,
        11,  13,  18,  18,  12,  17,  12,  22,  19,  18,  18,  37,  12,
         3,   9,  13,  10,   7,   7,  34,  24,  28,  12,  31,  26,  28,
        16,   7,  18,   8,   7,  13,  12,   9,   4,   1,   6,   5,  33,
        31,  22,  24,  13,  27,  13,   5,  10,   3,   6,   8,   5,  17,
        19,  19,  32,  25,  21,  22,  22,  10,  20,  28,  57,   5,   2,
         3,   7,  11,  11,  36,  16,  14,  10,   7,  11,  14,   